# Modelling charging profiles for different grid tariff structure

The first step entails that we model EV charging profiles for each charging station for different grid tariff structures. We consider two scenarios for each tariff structure: a scenario with fixed retail electricity prices and a scenario with dynamic retail electricity prices. The charging profiles are determined for different tariff values for each tariff structure. In a later step, the tariff value that corresponds with cost-recovery is determined for each EV adoption rate.

## Importing packages and loading data
As a first step, packages are imported and required datasets are loaded. In this notebook, we run the model using a sample EV charging session dataset, containing generated charging session data for two EV charging stations, as we do not have permission to share all used EV data. We also define the _'charging profile dict'_, in which the charging profiles for each tariff structure are saved.  

In [2]:
import pandas as pd
import datetime
import pytz
import numpy as np
charging_session_data=pd.read_pickle('data/charging_session_data_sample.pkl')
start_date=datetime.datetime(2022,1,1,tzinfo=pytz.timezone('CET'))
end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
timesteplist=pd.date_range(start=start_date,end=end_date+datetime.timedelta(days=2),freq='15Min',tz='CET')
day_ahead_prices=pd.read_pickle('data/day_ahead_market_prices_NL.pkl')
charging_profile_dict={}

## Fixed and flat volumetric grid tariffs
The fixed and flat volumetric tariff structures, which provide no incentive for adjusting EV charging patterns, serve as a baseline. In this section of the notebook, we model the baseline charging patterns. Under fixed retail prices, it is assumed that uncontrolled charging is applied for EV charging, while under dynamic retail prices, charging profiles are solely optimized for retail prices. The charging models associated with these tariff structures are imported from the _helperfunctions_ folder.

In [2]:
from helperfunctions.uncontrolled_charging_model import uncontrolled_charging

uncontrolled_charging_total=uncontrolled_charging(charging_session_data,timesteplist)
print('Uncontrolled charging profiles for all charging stations modelled')
charging_profile_dict['Fixed_tariffs_fixed']=uncontrolled_charging_total 

Uncontrolled charging profiles for all charging stations modelled


In [18]:
from helperfunctions.volumetric_ToU_model import volumetric_ToU

grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)']) #uniform grid tariff to ensure that tariff values do not provide incentive to adjust charging profiles
DA_charging_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
for CS in charging_session_data['Charging station ID'].unique():
    charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
    DA_charging_total[CS]=volumetric_ToU(charging_session_data_CS,timesteplist,CS,grid_tariff_df,day_ahead_prices,True)
    print('Optimal dynamic retail price charging profile for '+CS+' modelled')
charging_profile_dict['Fixed_tariffs_dynamic']=DA_charging_total 

Set parameter Username
Set parameter LicenseID to value 2656772
Academic license - for non-commercial use only - expires 2026-04-25
Optimal dynamic retail price charging profile for Charging station 1 modelled
Optimal dynamic retail price charging profile for Charging station 2 modelled


## Volumetric ToU tariffs
We model the volumetric ToU tariff structure for three considered tariff values. Please note that we considered many more tariff values for the analysis in this paper. As explained in the manuscript, we consider three tariff periods: low tariffs between 1:00 and 17:00, medium tariffs between 16:00-18:00 and 22:00-1:00 and high tariffs between 18:00-22:00. We also consider that the medium tariff value is twice low tariff value, and the high tariff value is 3 times the low tariff value. 

In [19]:
volumetric_tou_tariffvalues=[0.01,0.02,0.99]
charging_profile_dict['Volumetric_ToU_dynamic']={}
charging_profile_dict['Volumetric_ToU_fixed']={}

for low_tariff_value in volumetric_tou_tariffvalues:
    medium_tariff_value=low_tariff_value*2
    high_tariff_value=low_tariff_value*3
    grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
    high_tariff_hours=[18,19,20,21]
    medium_tariff_hours=[22,23,0,16,17]
    low_tariff_hours=list(range(1,16))
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(high_tariff_hours),high_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(medium_tariff_hours),medium_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(low_tariff_hours),low_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
    for dynamic_retail_prices_considered in [True, False]:
        volumetric_ToU_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            volumetric_ToU_total[CS]=volumetric_ToU(charging_session_data_CS,timesteplist,CS,grid_tariff_df,day_ahead_prices,dynamic_retail_prices_considered)
        if dynamic_retail_prices_considered==True:
            print('Volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for dynamic retail prices')
            charging_profile_dict['Volumetric_ToU_dynamic'][str(low_tariff_value)]=volumetric_ToU_total
        else:
            print('Volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for fixed retail prices')
            charging_profile_dict['Volumetric_ToU_fixed'][str(low_tariff_value)]=volumetric_ToU_total


Volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for fixed retail prices
Volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for fixed retail prices
Volumetric ToU profiles modelled for low tariff value of 0.99 €/kWh and for dynamic retail prices
Volumetric ToU profiles modelled for low tariff value of 0.99 €/kWh and for fixed retail prices


## Segmented Volumetric ToU Tariffs
As a next step, we model the charging profiles when considering segmented volumetric ToU tariffs. We again consider only a subset of the tariff values considered in the analysis. We consider the same number of tariffs and the hours for low, medium and high tariff hours as with volumetric ToU tariffs. The main difference is that a specific capacity is available for each tariff value at low, medium and high tariff hours. At low tariff hours, the low grid tariff applies for the first 4 kW of charging, and charging above this up to 8 kW is charged the medium grid tariff, and all extra charging the high grid tariff. At medium tariff hours, the medium grid tariff applies for the first 4 kW of charging, after which the high tariff applies for all charging above this value. At high tariff hours, the high grid tariff applies for all charging. 

In [5]:
from helperfunctions.segmented_volumetric_ToU_model import segmented_volumetric_ToU
segmented_volumetric_tou_tariffvalues=[0.01,0.02,0.03]
charging_profile_dict['Segmented_volumetric_ToU_dynamic']={}
charging_profile_dict['Segmented_volumetric_ToU_fixed']={}

for low_tariff_value in segmented_volumetric_tou_tariffvalues:
    medium_tariff_value=low_tariff_value*2
    high_tariff_value=low_tariff_value*3
    grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
    high_tariff_hours=[18,19,20,21]
    medium_tariff_hours=[22,23,0,16,17]
    low_tariff_hours=list(range(1,16))
    tariff_threshold_df=pd.DataFrame(index=timesteplist,columns=['Threshold_1','Threshold_2'])
    tariff_threshold_df['Threshold_1']=np.where(tariff_threshold_df.index.hour.isin(low_tariff_hours),4,0)
    tariff_threshold_df['Threshold_2']=np.where(tariff_threshold_df.index.hour.isin(high_tariff_hours),0,4)  
    for dynamic_retail_prices_considered in [True, False]:
        segmented_volumetric_ToU_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            segmented_volumetric_ToU_total[CS]=segmented_volumetric_ToU(charging_session_data_CS,timesteplist,CS,tariff_threshold_df,day_ahead_prices,dynamic_retail_prices_considered,low_tariff_value,medium_tariff_value,high_tariff_value)
        if dynamic_retail_prices_considered==True:
            print('Segmented volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for dynamic retail prices')
            charging_profile_dict['Segmented_volumetric_ToU_dynamic'][str(low_tariff_value)]=segmented_volumetric_ToU_total
        else:
            print('Segmented volumetric ToU profiles modelled for low tariff value of '+str(low_tariff_value)+' €/kWh and for fixed retail prices')
            charging_profile_dict['Segmented_volumetric_ToU_fixed'][str(low_tariff_value)]=segmented_volumetric_ToU_total

Segmented volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.01 €/kWh and for fixed retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.02 €/kWh and for fixed retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for dynamic retail prices
Segmented volumetric ToU profiles modelled for low tariff value of 0.03 €/kWh and for fixed retail prices


## Capacity tariffs
Modelling charging profiles under capacity tariffs requires a multi-step approach. The capacity model requires an estimated optimal capacity as an input. Therefore, we first determine for each month the optimal capacity levels for each tariff level when considering perfect foresight of the number of charging sessions and their arrival and departure times. These are saved in the _capacity prep dict_ and are used as input for the second step. 

In [6]:
from helperfunctions.capacity_preparation_model import capacity_tariffs_preparation
capacity_prep_dict={}
capacity_tariffvalues=[25,50,75]
for tariff_value in capacity_tariffvalues:
    for dynamic_retail_prices_considered in [True,False]:
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            if dynamic_retail_prices_considered==True:
                capacity_prep_dict[CS+'_'+str(tariff_value)+'_dynamic']=capacity_tariffs_preparation(charging_session_data_CS,timesteplist,day_ahead_prices,dynamic_retail_prices_considered,tariff_value)
                print('Optimal capacities determined for '+CS+' for tariff value of '+str(tariff_value)+' €/kW/year and for dynamic retail prices')
            else:
                capacity_prep_dict[CS+'_'+str(tariff_value)+'_fixed']=capacity_tariffs_preparation(charging_session_data_CS,timesteplist,day_ahead_prices,dynamic_retail_prices_considered,tariff_value)
                print('Optimal capacities determined for '+CS+' for tariff value of '+str(tariff_value)+' €/kW/year and for fixed retail prices')
pd.to_pickle(capacity_prep_dict,'results/optimal_capacities_capacity_tariff.pkl')

Optimal capacities determined for Charging station 1 for tariff value of 25 €/kW/year and for dynamic retail prices
Optimal capacities determined for Charging station 2 for tariff value of 25 €/kW/year and for dynamic retail prices
Optimal capacities determined for Charging station 1 for tariff value of 25 €/kW/year and for fixed retail prices
Optimal capacities determined for Charging station 2 for tariff value of 25 €/kW/year and for fixed retail prices
Optimal capacities determined for Charging station 1 for tariff value of 50 €/kW/year and for dynamic retail prices
Optimal capacities determined for Charging station 2 for tariff value of 50 €/kW/year and for dynamic retail prices
Optimal capacities determined for Charging station 1 for tariff value of 50 €/kW/year and for fixed retail prices
Optimal capacities determined for Charging station 2 for tariff value of 50 €/kW/year and for fixed retail prices
Optimal capacities determined for Charging station 1 for tariff value of 75 €/kW

In the second step, we model the charging patterns for the capacity tariffs without assuming perfect foresight regarding the number and characteristics of future charging sessions, ensuring the model does not anticipate end-of-month charging peaks at the beginning of the month. This model requires an estimate of the optimal charging peak as an input. In the analysis of this work, we based this estimate for each month on the average of the optimal charging peaks of the previous three months, determined in the previous step. Note that in this sample analysis, we did not determine the optimal capacities for the months before 2022, assuming an optimal peak of 10 kW for those months when modelling the charging patterns of the first three months of 2022. In the paper, we did determine optimal capacity for the months prior 2022. 

**Note that since we are considering a rolling optimization model, the modelling time is considerably longer**


In [7]:
from helperfunctions.capacity_model import capacity_tariffs
charging_profile_dict['Capacity_tariffs_dynamic']={}
charging_profile_dict['Capacity_tariffs_fixed']={}

capacity_prep_dict=pd.read_pickle('results/optimal_capacities_capacity_tariff.pkl')
for tariff_value in capacity_tariffvalues:
    for dynamic_retail_prices_considered in [True,False]:
        capacity_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
        for CS in charging_session_data['Charging station ID'].unique():
            charging_session_data_CS=charging_session_data[charging_session_data['Charging station ID']==CS]
            for month in range(1,13):
                initial_peak=0
                for prev_month in [month-1,month-2,month-3]:
                    if dynamic_retail_prices_considered==True:
                        try:
                            initial_peak+=capacity_prep_dict[CS+'_'+str(tariff_value)+'_dynamic'][prev_month]/3
                        except:
                            initial_peak+=10/3
                    else:
                        try:
                            initial_peak+=capacity_prep_dict[CS+'_'+str(tariff_value)+'_fixed'][prev_month]/3
                        except:
                            initial_peak+=10/3
                try:
                    timesteplist_month=pd.date_range(start=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET')),end=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))+datetime.timedelta(days=2),freq='15Min',tz='CET')
                except:
                    timesteplist_month=pd.date_range(start=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET')),end=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))+datetime.timedelta(days=2),freq='15Min',tz='CET')
                charging_session_data_CS_month=charging_session_data_CS[charging_session_data_CS['Arrival time'].dt.month==month]
                results_CS_month=capacity_tariffs(charging_session_data_CS_month, timesteplist_month, CS, day_ahead_prices, dynamic_retail_prices_considered, tariff_value,initial_peak)
                for t in timesteplist_month:
                    capacity_total.at[t,CS]+=results_CS_month.loc[t,CS]

                if dynamic_retail_prices_considered==True:
                    print('Capacity tariff profiles modelled for '+CS+' for tariff value of '+str(tariff_value)+' €/kW for month '+str(month)+' and for dynamic retail prices')
                else:
                    print('Capacity tariff profiles modelled for '+CS+' for tariff value of '+str(tariff_value)+' €/kW for month '+str(month)+' and for fixed retail prices')
        if dynamic_retail_prices_considered==True:
            charging_profile_dict['Capacity_tariffs_dynamic'][str(tariff_value)]=capacity_total
        else:
            charging_profile_dict['Capacity_tariffs_fixed'][str(tariff_value)]=capacity_total
            


Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 1 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 2 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 3 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 4 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 5 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 6 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 7 and for dynamic retail prices
Capacity tariff profiles modelled for Charging station 1 for tariff value of 25 €/kW for month 8 and for dynami

## Capacity-subscription tariffs
For this tariff structure, we follow a three-step approach. In the first step, we determine the optimal charging profiles for different subscribed capacities for each considered charging stations, thereby considering a fixed exceedance fee of 0.3 €/kWh. Note that in the analysis we performed for this paper, more subscribed capacity values were considered. 

In [51]:
from helperfunctions.capacity_subscription_model import capacity_subscription
exceedance_fee=0.3
capacity_subscription_dict={}
for dynamic_retail_prices_considered in [True,False]:
    if dynamic_retail_prices_considered==True:
        capacity_subscription_dict['dynamic']={}
    else:
        capacity_subscription_dict['fixed']={}
    capacity_subscription_total=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
    subscribed_capacity_list=[4,8,12,16]
    for CS in charging_session_data['Charging station ID'].unique():
        capacity_subscription_CS=pd.DataFrame(0,index=timesteplist,columns=subscribed_capacity_list)
        for subscribed_capacity in subscribed_capacity_list:
            resultdf_CS_subscribed_capacity=capacity_subscription(charging_session_data, timesteplist, CS, day_ahead_prices, dynamic_retail_prices_considered, exceedance_fee, subscribed_capacity)
            capacity_subscription_CS[subscribed_capacity]=resultdf_CS_subscribed_capacity[CS].copy()
            if dynamic_retail_prices_considered==True:
                print('Capacity-subscription profiles modelled for '+CS+' for subscribed capacity of '+str(subscribed_capacity)+' kW for dynamic retail prices')
            else:
                print('Capacity-subscription profiles modelled for '+CS+' for subscribed capacity of '+str(subscribed_capacity)+' kW for fixed retail prices')
        if dynamic_retail_prices_considered==True:
            capacity_subscription_dict['dynamic'][CS]=capacity_subscription_CS
        else:
            capacity_subscription_dict['fixed'][CS]=capacity_subscription_CS    
            

Capacity-subscription profiles modelled for Charging station 1 for subscribed capacity of 4 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 1 for subscribed capacity of 8 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 1 for subscribed capacity of 12 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 1 for subscribed capacity of 16 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 2 for subscribed capacity of 4 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 2 for subscribed capacity of 8 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 2 for subscribed capacity of 12 kW for dynamic retail prices
Capacity-subscription profiles modelled for Charging station 2 for subscribed capacity of 16 kW for dynamic retail prices
Capacity-subscription profil

In the second step, we considered different capacity-subscription tariff levels (€/kW/year) to find the optimal subscribed capacity for each month for each charging station for each tariff level. We first create a dictionary _capacity subscription cost dict_ in which we determine for each tariff level and subscribed capacity the total costs per charging station (grid tariff costs + retail electricity costs).

In [52]:
capacity_subscription_cost_dict={}
capacity_subscription_tariffvalues=[25.0,50.0,75.0]
for dynamic_retail_prices_considered in [True,False]:
    if dynamic_retail_prices_considered==True:
        capacity_subscription_cost_dict['dynamic']={}
    else:
        capacity_subscription_cost_dict['fixed']={}
    for CS in charging_session_data['Charging station ID'].unique():
        if dynamic_retail_prices_considered==True:
            capacity_subscription_cost_dict['dynamic'][CS]={}
        else:
            capacity_subscription_cost_dict['fixed'][CS]={}
        for tariff_value in capacity_subscription_tariffvalues:
            if dynamic_retail_prices_considered==True:
                capacity_subscription_cost_dict['dynamic'][CS][tariff_value]=pd.DataFrame(0,index=range(1,13),columns=subscribed_capacity_list)
            else:
                capacity_subscription_cost_dict['fixed'][CS][tariff_value]=pd.DataFrame(0,index=range(1,13),columns=subscribed_capacity_list)
            for subscribed_capacity in subscribed_capacity_list:
                if dynamic_retail_prices_considered==True:
                    charging_profile_CS=pd.DataFrame(capacity_subscription_dict['dynamic'][CS][subscribed_capacity])
                else:
                    charging_profile_CS=pd.DataFrame(capacity_subscription_dict['fixed'][CS][subscribed_capacity])
                for month in range(1,13):
                    start_date=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET'))
                    try:
                        end_date=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))
                    except:
                        end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
                    charging_profile_CS_month=charging_profile_CS[(charging_profile_CS.index>=start_date) & (charging_profile_CS.index<end_date)]
                    charging_profile_CS_month['exceedence']=np.where(charging_profile_CS_month[subscribed_capacity]>subscribed_capacity,charging_profile_CS_month[subscribed_capacity]-subscribed_capacity,0)
                    exceedance_month=charging_profile_CS_month['exceedence'].sum()*0.25
                    total_grid_costs_month=exceedance_month*exceedance_fee+subscribed_capacity*tariff_value/12
                    if dynamic_retail_prices_considered==True:
                        day_ahead_prices_month=day_ahead_prices[(day_ahead_prices.index>=start_date) & (day_ahead_prices.index<end_date)]
                        charging_profile_CS_month['retail costs']=charging_profile_CS_month[subscribed_capacity]*day_ahead_prices_month['Day-ahead price (€/MWh)']/1000*0.25
                        retail_price_month=charging_profile_CS_month['retail costs'].sum()
                        capacity_subscription_cost_dict['dynamic'][CS][tariff_value].at[month,subscribed_capacity]=total_grid_costs_month+retail_price_month
                    else:
                        capacity_subscription_cost_dict['fixed'][CS][tariff_value].at[month,subscribed_capacity]=total_grid_costs_month

In our model, we base the chosen subscribed capacity level at the beginning of each month on the capacity levels that would have yielded lowest costs in the previous three months. Given this, we determine the charging profiles for a specific tariff value by choosing the subscribed capacity value for each month, and then taking the charging profiles corresponding to this subscribed capacity. Note that in this sample analysis, we did not determine the profiles for the months before 2022, assuming an optimal subscribed capacity of 12 kW for the first three months of 2022. In the paper, we did determine optimal capacity for the months prior 2022. 

In [53]:
charging_profile_dict['Capacity_subscription_dynamic']={}
charging_profile_dict['Capacity_subscription_fixed']={}
subscribed_capacity_dict={}
for tariff_value in capacity_subscription_tariffvalues:
    subscribed_capacity_dict[tariff_value]={}
    for dynamic_retail_prices_considered in [True,False]:
        if dynamic_retail_prices_considered==True:
            charging_profile_dict['Capacity_subscription_dynamic'][str(tariff_value)]=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
            subscribed_capacity_dict[tariff_value]['dynamic']=pd.DataFrame(index=range(1,13),columns=charging_session_data['Charging station ID'].unique())
        else:
            charging_profile_dict['Capacity_subscription_fixed'][str(tariff_value)]=pd.DataFrame(0,index=timesteplist,columns=charging_session_data['Charging station ID'].unique())
            subscribed_capacity_dict[tariff_value]['fixed']=pd.DataFrame(index=range(1,13),columns=charging_session_data['Charging station ID'].unique())

        for CS in charging_session_data['Charging station ID'].unique():
            for month in range(1,13):
                try:
                    timesteplist_month=pd.date_range(start=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET')),end=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET')))
                except:
                    timesteplist_month=pd.date_range(start=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET')),end=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET')))
                if month <=3:
                    optimal_subscribed_capacity=12 #only for this sample analysis, see explanation above
                else:
                    if dynamic_retail_prices_considered==True:
                        cost_values=capacity_subscription_cost_dict['dynamic'][CS][tariff_value]
                    else:
                        cost_values=capacity_subscription_cost_dict['fixed'][CS][tariff_value]
                    cost_values=cost_values.loc[[month-1,month-2,month-3],:]
                    optimal_subscribed_capacity= cost_values.sum().idxmin()
                if dynamic_retail_prices_considered==True:
                    subscribed_capacity_dict[tariff_value]['dynamic'].at[month,CS]=optimal_subscribed_capacity
                else:
                    subscribed_capacity_dict[tariff_value]['fixed'].at[month,CS]=optimal_subscribed_capacity
                for t in timesteplist_month:
                    if dynamic_retail_prices_considered==True:
                        charging_profile_dict['Capacity_subscription_dynamic'][str(tariff_value)].at[t,CS]+=capacity_subscription_dict['dynamic'][CS].loc[t,optimal_subscribed_capacity]
                    else:
                        charging_profile_dict['Capacity_subscription_fixed'][str(tariff_value)].at[t,CS]+=capacity_subscription_dict['fixed'][CS].loc[t,optimal_subscribed_capacity]
pd.to_pickle(subscribed_capacity_dict,'results/subscribed_capacities_capacity_subscription.pkl')
pd.to_pickle(charging_profile_dict,'results/charging_profiles_alltariffvalues.pkl')

# Determining tariff values
After modelling the EV charging profiles for a range of tariff values for each grid tariff structure, we determine the tariff value that most closely leads to cost-recovery. For each considered EV adoption rate, we first determine the grid costs based on the aggregate demand peak. After this, we determine the total grid tariff income for the grid operator, and determine which grid tariff value leads to the closest alignment between grid costs and tariff income. We generate three dictionaries, showing the grid costs, tariff values and profiles for each considered scenario.

**Note that since we only consider two charging stations and a limited number of tariff values in this sample analysis, the projected profiles and tariff values are not realistic!**

In [54]:
def cost_calc_segmented_volumetric_ToU(profile_df,CS,low_tariff_value,medium_tariff_value,high_tariff_value,threshold_df):
    profile_df['threshold_1']=threshold_df['Threshold_1'].copy()
    profile_df['threshold_2']=threshold_df['Threshold_2'].copy()
    profile_df['low_tariff_demand']=np.where(profile_df['threshold_1']>0,profile_df[['threshold_1',CS]].min(axis=1),0)
    profile_df['remaining_demand']=profile_df[CS]-profile_df['low_tariff_demand']
    profile_df['medium_tariff_demand']=np.where(profile_df['threshold_2']>0,profile_df[['threshold_2','remaining_demand']].min(axis=1),0)
    profile_df['high_tariff_demand']=profile_df[CS]-profile_df['low_tariff_demand']-profile_df['medium_tariff_demand']
    total_costs=profile_df['low_tariff_demand'].sum()*low_tariff_value*0.25+profile_df['medium_tariff_demand'].sum()*medium_tariff_value*0.25+profile_df['high_tariff_demand'].sum()*high_tariff_value*0.25
    return total_costs

charging_profile_dict=pd.read_pickle('results/charging_profiles_alltariffvalues.pkl')
hh_profiles=pd.read_pickle('data/household_profiles.pkl')
hh_profiles_nonnegative=hh_profiles.copy()
hh_profiles_nonnegative[hh_profiles_nonnegative<0]=0 #set negative values to 0, as we do not consider net metering for volumetric tariff structures in this analysis
total_load_dict={}
total_grid_cost_dict={}
tariff_value_dict={}
car_possession_rate_NL=1.078
no_cars_case_study=len(hh_profiles.columns)*car_possession_rate_NL
total_charging_demand_case_study=no_cars_case_study*12000*0.188/np.sqrt(0.87) #average annual mileage of 12000 km/year, average roundtrip charging/discharging efficiency of 87%, average driving efficiency of 0.188 kWh/km
no_CS_100=total_charging_demand_case_study/11368 #number of charging stations at 100% EV adoption rate, 11368 kWh/year is the average annual charging demand per charging station in the sample data
expansion_cost_value=120 #€/kW
for EV_adoption_rate in [0.25,0.5,0.75,1.0]:
    no_CS=no_CS_100*EV_adoption_rate #number of charging stations at EV adoption rate
    tariff_value_dict[EV_adoption_rate]={}
    total_load_dict[EV_adoption_rate]={}
    total_grid_cost_dict[EV_adoption_rate]={}
    for retail_electricity_prices in ['dynamic','fixed']:
        tariff_value_dict[EV_adoption_rate][retail_electricity_prices]=pd.DataFrame(index=['Volumetric_ToU','Segmented_volumetric_ToU','Capacity_tariffs','Capacity_subscription'],columns=['Tariff value'])
        total_load_dict[EV_adoption_rate][retail_electricity_prices]=pd.DataFrame(index=timesteplist,columns=['Volumetric_ToU','Segmented_volumetric_ToU','Capacity_tariffs','Capacity_subscription'])
        total_grid_cost_dict[EV_adoption_rate][retail_electricity_prices]=pd.DataFrame(index=['Volumetric_ToU','Segmented_volumetric_ToU','Capacity_tariffs','Capacity_subscription'],columns=['Grid costs'])
        
        "Volumetric ToU"
        tariff_income_grid_cost_df=pd.DataFrame(0,index=charging_profile_dict['Volumetric_ToU_'+retail_electricity_prices].keys(),columns=['Tariff income','Grid costs'])
        for low_tariff_value in charging_profile_dict['Volumetric_ToU_'+retail_electricity_prices].keys():
            low_tariff_value=float(low_tariff_value)
            "Determine grid costs"
            EV_charging_profile=charging_profile_dict['Volumetric_ToU_'+retail_electricity_prices][str(low_tariff_value)] #select charging profile for the given tariff value
            EV_charging_profile['Mean']=EV_charging_profile.mean(axis=1) #average charging profile for all charging stations
            total_load_df=pd.DataFrame(0,index=timesteplist,columns=['Household load (kW)','EV charging load (kW)'])
            total_load_df['EV charging load (kW)']=EV_charging_profile['Mean'].copy()*no_CS #total EV charging load equals average charging profile multiplied by number of charging stations
            total_load_df['Household load (kW)']=hh_profiles.sum(axis=1).copy()
            total_load_df['Total load (kW)']=total_load_df['Household load (kW)']+total_load_df['EV charging load (kW)']
            total_load_df=total_load_df[total_load_df.index<datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))]
            peak_demand=total_load_df['Total load (kW)'].max()
            total_expansion_costs=(peak_demand-hh_profiles.sum(axis=1).max())*expansion_cost_value #total expansion costs are equal to the peak demand minus the initial peak demand (which is equal to the household load) multiplied by the expansion cost value
            tariff_income_grid_cost_df.at[str(low_tariff_value),'Grid costs']=total_expansion_costs #store grid costs in the dataframe
            "Determine tariff income"
            medium_tariff_value=low_tariff_value*2
            high_tariff_value=low_tariff_value*3
            grid_tariff_df=pd.DataFrame(0,index=timesteplist,columns=['Grid tariff (€/kWh)'])
            high_tariff_hours=[18,19,20,21]
            medium_tariff_hours=[22,23,0,16,17]
            low_tariff_hours=list(range(1,16))
            grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(high_tariff_hours),high_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
            grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(medium_tariff_hours),medium_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
            grid_tariff_df['Grid tariff (€/kWh)']=np.where(grid_tariff_df.index.hour.isin(low_tariff_hours),low_tariff_value,grid_tariff_df['Grid tariff (€/kWh)'])
            total_tariff_income=0
            for hh in hh_profiles_nonnegative.columns:
                total_tariff_income+=(hh_profiles_nonnegative[hh]*grid_tariff_df['Grid tariff (€/kWh)']).sum()*0.25
            total_tariff_income+=(EV_charging_profile['Mean']*no_CS*grid_tariff_df['Grid tariff (€/kWh)']).sum()*0.25
            tariff_income_grid_cost_df.at[str(low_tariff_value),'Tariff income']=total_tariff_income
        tariff_income_grid_cost_df['Abs_difference']=abs(tariff_income_grid_cost_df['Tariff income']-tariff_income_grid_cost_df['Grid costs'])
        selected_tariff_value=tariff_income_grid_cost_df['Abs_difference'].idxmin() #select tariff value with minimum difference between tariff income and grid costs
        tariff_value_dict[EV_adoption_rate][retail_electricity_prices].at['Volumetric_ToU','Tariff value']=float(selected_tariff_value)
        total_load_dict[EV_adoption_rate][retail_electricity_prices]['Volumetric_ToU']=total_load_df['Total load (kW)'].copy()
        total_grid_cost_dict[EV_adoption_rate][retail_electricity_prices].at['Volumetric_ToU','Grid costs']=tariff_income_grid_cost_df['Grid costs'].loc[selected_tariff_value].copy()
    
        "Segmented volumetric ToU"
        tariff_income_grid_cost_df=pd.DataFrame(0,index=charging_profile_dict['Segmented_volumetric_ToU_'+retail_electricity_prices].keys(),columns=['Tariff income','Grid costs'])
        for low_tariff_value in charging_profile_dict['Segmented_volumetric_ToU_'+retail_electricity_prices].keys():
            low_tariff_value=float(low_tariff_value)
            "Determine grid costs"
            EV_charging_profile=charging_profile_dict['Segmented_volumetric_ToU_'+retail_electricity_prices][str(low_tariff_value)] #select charging profile for the given tariff value
            EV_charging_profile['Mean']=EV_charging_profile.mean(axis=1) #average charging profile for all charging stations
            total_load_df=pd.DataFrame(0,index=timesteplist,columns=['Household load (kW)','EV charging load (kW)'])
            total_load_df['EV charging load (kW)']=EV_charging_profile['Mean'].copy()*no_CS #total EV charging load equals average charging profile multiplied by number of charging stations
            total_load_df['Household load (kW)']=hh_profiles.sum(axis=1).copy()
            total_load_df['Total load (kW)']=total_load_df['Household load (kW)']+total_load_df['EV charging load (kW)']
            total_load_df=total_load_df[total_load_df.index<datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))]
            peak_demand=total_load_df['Total load (kW)'].max()
            total_expansion_costs=(peak_demand-hh_profiles.sum(axis=1).max())*expansion_cost_value #total expansion costs are equal to the peak demand minus the initial peak demand (which is equal to the household load) multiplied by the expansion cost value
            tariff_income_grid_cost_df.at[str(low_tariff_value),'Grid costs']=total_expansion_costs #store grid costs in the dataframe
            del EV_charging_profile['Mean']
            "Determine tariff income"
            medium_tariff_value=low_tariff_value*2
            high_tariff_value=low_tariff_value*3
            high_tariff_hours=[18,19,20,21]
            medium_tariff_hours=[22,23,0,16,17]
            low_tariff_hours=list(range(1,16))
            tariff_threshold_df=pd.DataFrame(index=timesteplist,columns=['Threshold_1','Threshold_2'])
            tariff_threshold_df['Threshold_1']=np.where(tariff_threshold_df.index.hour.isin(low_tariff_hours),4,0)
            tariff_threshold_df['Threshold_2']=np.where(tariff_threshold_df.index.hour.isin(high_tariff_hours),0,4) 
            total_tariff_income=0
            for hh in hh_profiles_nonnegative.columns:
                total_tariff_income+=cost_calc_segmented_volumetric_ToU(hh_profiles_nonnegative,hh,low_tariff_value,medium_tariff_value,high_tariff_value,tariff_threshold_df)
            total_tariff_income_CS=0
            for CS in EV_charging_profile: 
                total_tariff_income_CS+=cost_calc_segmented_volumetric_ToU(EV_charging_profile,CS,low_tariff_value,medium_tariff_value,high_tariff_value,tariff_threshold_df)
            total_tariff_income+=total_tariff_income_CS/len(EV_charging_profile.columns)*no_CS #total tariff income from EV charging is equal to average tariff income per modelled charging stations times the number of expected charging stations
            tariff_income_grid_cost_df.at[str(low_tariff_value),'Tariff income']=total_tariff_income
        tariff_income_grid_cost_df['Abs_difference']=abs(tariff_income_grid_cost_df['Tariff income']-tariff_income_grid_cost_df['Grid costs'])
        selected_tariff_value=tariff_income_grid_cost_df['Abs_difference'].idxmin() #select tariff value with minimum difference between tariff income and grid costs
        tariff_value_dict[EV_adoption_rate][retail_electricity_prices].at['Segmented_volumetric_ToU','Tariff value']=float(selected_tariff_value)
        total_load_dict[EV_adoption_rate][retail_electricity_prices]['Segmented_volumetric_ToU']=total_load_df['Total load (kW)'].copy()
        total_grid_cost_dict[EV_adoption_rate][retail_electricity_prices].at['Segmented_volumetric_ToU','Grid costs']=tariff_income_grid_cost_df['Grid costs'].loc[selected_tariff_value].copy()

        "Capacity tariffs"
        tariff_income_grid_cost_df=pd.DataFrame(0,index=charging_profile_dict['Capacity_tariffs_'+retail_electricity_prices].keys(),columns=['Tariff income','Grid costs'])
        for tariff_value in charging_profile_dict['Capacity_tariffs_'+retail_electricity_prices].keys():
            tariff_value=int(tariff_value)
            "Determine grid costs"
            EV_charging_profile=charging_profile_dict['Capacity_tariffs_'+retail_electricity_prices][str(tariff_value)]
            EV_charging_profile['Mean']=EV_charging_profile.mean(axis=1)
            total_load_df['EV charging load (kW)']=EV_charging_profile['Mean'].copy()*no_CS #total EV charging load equals average charging profile multiplied by number of charging stations
            total_load_df['Household load (kW)']=hh_profiles.sum(axis=1).copy()
            total_load_df['Total load (kW)']=total_load_df['Household load (kW)']+total_load_df['EV charging load (kW)']
            total_load_df=total_load_df[total_load_df.index<datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))]
            peak_demand=total_load_df['Total load (kW)'].max()
            total_expansion_costs=(peak_demand-hh_profiles.sum(axis=1).max())*expansion_cost_value #total expansion costs are equal to the peak demand minus the initial peak demand (which is equal to the household load) multiplied by the expansion cost value
            tariff_income_grid_cost_df.at[str(tariff_value),'Grid costs']=total_expansion_costs #store grid costs in the dataframe
            del EV_charging_profile['Mean']
            "Determine tariff income"
            total_tariff_income=0
            for hh in hh_profiles.columns:
                avg_peak=0
                for month in range(1,13):
                    start_date=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET'))
                    try:
                        end_date=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))
                    except:
                        end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
                    hh_profiles_month=hh_profiles[(hh_profiles.index>=start_date)&(hh_profiles.index<end_date)]
                    avg_peak+=hh_profiles_month[hh].max()/12
                total_tariff_income+=avg_peak*tariff_value
            for CS in EV_charging_profile.columns:
                avg_peak=0
                for month in range(1,13):
                    start_date=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET'))
                    try:
                        end_date=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))
                    except:
                        end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
                    EV_charging_profile_month=EV_charging_profile[(EV_charging_profile.index>=start_date)&(EV_charging_profile.index<end_date)]
                    avg_peak+=EV_charging_profile_month[CS].max()/12
                total_tariff_income+=avg_peak*tariff_value*no_CS/len(EV_charging_profile.columns)
            tariff_income_grid_cost_df.at[str(tariff_value),'Tariff income']=total_tariff_income
        tariff_income_grid_cost_df['Abs_difference']=abs(tariff_income_grid_cost_df['Tariff income']-tariff_income_grid_cost_df['Grid costs'])
        selected_tariff_value=tariff_income_grid_cost_df['Abs_difference'].idxmin()
        #select tariff value with minimum difference between tariff income and grid costs
        tariff_value_dict[EV_adoption_rate][retail_electricity_prices].at['Capacity_tariffs','Tariff value']=int(selected_tariff_value)
        total_load_dict[EV_adoption_rate][retail_electricity_prices]['Capacity_tariffs']=total_load_df['Total load (kW)'].copy()
        total_grid_cost_dict[EV_adoption_rate][retail_electricity_prices].at['Capacity_tariffs','Grid costs']=tariff_income_grid_cost_df['Grid costs'].loc[selected_tariff_value].copy()

        "Capacity subscription"
        tariff_income_grid_cost_df=pd.DataFrame(0,index=charging_profile_dict['Capacity_subscription_'+retail_electricity_prices].keys(),columns=['Tariff income','Grid costs'])
        subscribed_capacity_dict=pd.read_pickle('results/subscribed_capacities_capacity_subscription.pkl')
        for tariff_value in charging_profile_dict['Capacity_subscription_'+retail_electricity_prices].keys():
            tariff_value=float(tariff_value)
            "Determine grid costs"
            EV_charging_profile=charging_profile_dict['Capacity_subscription_'+retail_electricity_prices][str(tariff_value)]
            EV_charging_profile['Mean']=EV_charging_profile.mean(axis=1)
            total_load_df['EV charging load (kW)']=EV_charging_profile['Mean'].copy()*no_CS
            total_load_df['Household load (kW)']=hh_profiles.sum(axis=1).copy()
            total_load_df['Total load (kW)']=total_load_df['Household load (kW)']+total_load_df['EV charging load (kW)']
            total_load_df=total_load_df[total_load_df.index<datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))]
            peak_demand=total_load_df['Total load (kW)'].max()
            total_expansion_costs=(peak_demand-hh_profiles.sum(axis=1).max())*expansion_cost_value
            tariff_income_grid_cost_df.at[str(tariff_value),'Grid costs']=total_expansion_costs #store grid costs in the dataframe
            del EV_charging_profile['Mean']
            "Determine tariff income"
            total_tariff_income=0
            for hh in hh_profiles.columns:
                for month in range(1,13):
                    subscribed_capacity=2 #default assumed subscribed capacity for households
                    start_date=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET'))
                    try:
                        end_date=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))
                    except:
                        end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
                    hh_profiles_month=hh_profiles[(hh_profiles.index>=start_date)&(hh_profiles.index<end_date)]
                    hh_profiles_month['exceedance']=np.where(hh_profiles_month[hh]>subscribed_capacity,hh_profiles_month[hh]-subscribed_capacity,0)
                    exceedance_month=hh_profiles_month['exceedance'].sum()*0.25
                    total_tariff_income+=subscribed_capacity*tariff_value/12+exceedance_month*exceedance_fee
            total_tariff_income_CS=0
            for CS in EV_charging_profile.columns:
                for month in range(1,13):
                    subscribed_capacity=subscribed_capacity_dict[tariff_value][retail_electricity_prices].loc[month,CS]
                    start_date=datetime.datetime(2022,month,1,tzinfo=pytz.timezone('CET'))
                    try:
                        end_date=datetime.datetime(2022,month+1,1,tzinfo=pytz.timezone('CET'))
                    except:
                        end_date=datetime.datetime(2023,1,1,tzinfo=pytz.timezone('CET'))
                    EV_charging_profile_month=EV_charging_profile[(EV_charging_profile.index>=start_date)&(EV_charging_profile.index<end_date)]
                    EV_charging_profile_month['exceedance']=np.where(EV_charging_profile_month[CS]>subscribed_capacity,EV_charging_profile_month[CS]-subscribed_capacity,0)
                    exceedance_month=EV_charging_profile_month['exceedance'].sum()*0.25
                    total_tariff_income_CS+=subscribed_capacity*tariff_value/12+exceedance_month*exceedance_fee
            total_tariff_income+=total_tariff_income_CS/len(EV_charging_profile.columns)*no_CS #total tariff income from EV charging is equal to average tariff income per modelled charging stations times the number of expected charging stations
            tariff_income_grid_cost_df.at[str(tariff_value),'Tariff income']=total_tariff_income
        tariff_income_grid_cost_df['Abs_difference']=abs(tariff_income_grid_cost_df['Tariff income']-tariff_income_grid_cost_df['Grid costs'])
        selected_tariff_value=tariff_income_grid_cost_df['Abs_difference'].idxmin() #select tariff value with minimum difference between tariff income and grid costs
        tariff_value_dict[EV_adoption_rate][retail_electricity_prices].at['Capacity_subscription','Tariff value']=float(selected_tariff_value)
        total_load_dict[EV_adoption_rate][retail_electricity_prices]['Capacity_subscription']=total_load_df['Total load (kW)'].copy()
        total_grid_cost_dict[EV_adoption_rate][retail_electricity_prices].at['Capacity_subscription','Grid costs']=tariff_income_grid_cost_df['Grid costs'].loc[selected_tariff_value].copy()
pd.to_pickle(tariff_value_dict,'results/tariff_values.pkl')
pd.to_pickle(total_load_dict,'results/total_loads.pkl')
pd.to_pickle(total_grid_cost_dict,'results/total_grid_costs.pkl')
                    

        

KeyboardInterrupt: 